# Project: Performance of phenotype algorithms for the identification of opioid-exposed infants, Andrew D. Wiese et al. Hospital Pediatrics 2024
# Title: Exclude infants with evidence of critical illness
# Summary: 
## Exclude infants with evidence of critical illness based on CPT codes

# Notes:
- Look for evidence of critical illness during birth hospitalization stay


##### Algorithm steps:

```
1. Define critical illness CPT codes: 
   - '99468', '99469', '99291', '99292'

2. Get baby procedure records from procedure_occurrence table for critical codes

3. Join with first visit table on baby_person_id

4. Filter records where procedure date is between first visit start and end dates 

5. Register final table as global temp view
```

##### Data Dictionaries:

**critical4cpt_code_list** - List of critical illness CPT codes 

**proc_df** - Baby procedure records filtered by critical codes

**mom_baby_step1_baby1stvisit** - Table containing baby's first visit details

**mom_baby_step3_critical4cpt** - Final output table 

##### Usage Notes:
```
- This generates a table of critical illness procedure codes for each baby that occurred during the first visit
- It filters the procedure records to only include those within the start/end dates of the first visit
- The final table is registered as a global temp view for easy access in subsequent steps
- Validation should be performed on the final table to ensure accuracy
```

In [0]:
%run "./project_modules"

##### 1. Critical illness CPT codes: '99468', '99469', '99291', '99292'
##### 2. procedure_occurrence 
##### 3. only for baby's code


In [0]:
proc_df=mon_baby_records_code_list('procedure','fact_id_2',critical4cpt_code_list)
proc_df.createOrReplaceTempView("proc_df")

sql="""
  
    select  distinct proc_df.*,c.first_visit_start_date,c.first_visit_end_date from 
    proc_df
    inner join 
    global_temp.mom_baby_step1_baby1stvisit c 
    on proc_df.baby_person_id = c.baby_person_id
    where code_date >= first_visit_start_date and code_date <= first_visit_end_date;
    """
mom_baby_step3_critical4cpt = spark.sql(sql)
mom_baby_step3_critical4cpt.name='mom_baby_step3_critical4cpt'
register_parquet_global_view(mom_baby_step3_critical4cpt)

##### Validation

In [0]:
df_inspection("global_temp.mom_baby_step3_critical4cpt","all")